In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import warnings 
# import geopandas as gpd
import numpy as np
# from shapely.geometry import Polygon, LineString, Point
warnings.filterwarnings(action='ignore')

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
from sklearn.preprocessing import StandardScaler

In [3]:
plt.rcParams['font.family'] = 'Malgun Gothic'

## 경감지수 전처리

### 학교정보 및 상담 데이터

In [4]:
# 필요한 파일 출력
councel = pd.read_csv('서울시고등학교_교육여건_상담.csv', encoding='cp949')
school_info = pd.read_csv('서울시고등학교_기본정보.csv', encoding='cp949')
police_loc = pd.read_csv('경찰청_경찰관서 위치.csv', encoding='cp949')

In [5]:
print(councel.info())
print(school_info.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   구                       320 non-null    object 
 1   학교명                     320 non-null    object 
 2   학교코드                    320 non-null    object 
 3   내부상담_실시여부               320 non-null    int64  
 4   외부상담_실시여부               320 non-null    int64  
 5   wee클래스_운영여부             320 non-null    int64  
 6   학교폭력_예방교육_평균교육시간(1학급당)  320 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 17.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   지역              320 non-null    object 
 1   학교코드            320 non-null    object 
 2   학교명             320 non-null    object 
 3   설립구분            

In [6]:
# 학교 기본정보 + 교육여건_상담 데이터 병합
school_info = school_info[['학교코드','위도','경도','법정동코드']]
school = pd.merge(councel, school_info, on='학교코드', how='left')
school.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   구                       320 non-null    object 
 1   학교명                     320 non-null    object 
 2   학교코드                    320 non-null    object 
 3   내부상담_실시여부               320 non-null    int64  
 4   외부상담_실시여부               320 non-null    int64  
 5   wee클래스_운영여부             320 non-null    int64  
 6   학교폭력_예방교육_평균교육시간(1학급당)  320 non-null    float64
 7   위도                      320 non-null    float64
 8   경도                      320 non-null    float64
 9   법정동코드                   320 non-null    int64  
dtypes: float64(3), int64(4), object(3)
memory usage: 27.5+ KB


In [7]:
police_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   청       274 non-null    object 
 1   서       274 non-null    object 
 2   지구대파출소  274 non-null    object 
 3   X좌표     274 non-null    float64
 4   Y좌표     274 non-null    float64
dtypes: float64(2), object(3)
memory usage: 10.8+ KB


In [8]:
Police_station = police_loc.rename({'X좌표':'경도','Y좌표':'위도'},axis=1)
Police_station

,청,서,지구대파출소,경도,위도
0,서울청,서울강남경찰서,논현1파출소,127.029316,37.513791
1,서울청,서울강남경찰서,논현2파출소,127.033875,37.515220
2,서울청,서울강남경찰서,삼성1파출소,127.060440,37.514852
3,서울청,서울강남경찰서,삼성2파출소,127.048063,37.511289
4,서울청,서울강남경찰서,서울강남경찰서,127.067177,37.509036
...,...,...,...,...,...
269,서울청,서울혜화경찰서,종로5가파출소,127.001606,37.570727
270,서울청,서울혜화경찰서,창신파출소,127.013702,37.578482
271,서울청,서울혜화경찰서,혜화파출소,127.001422,37.586135
272,서울청,서울혜화경찰서,효제파출소,127.003606,37.574259


In [9]:
# !pip install haversine

### 학교 500m 내 경찰관서수

In [10]:
from tqdm.notebook import tqdm
from tqdm import tqdm, tqdm_notebook

from haversine import haversine

In [11]:
# 경찰관서수 구하는 함수 
def close_ps(school, Police_station, column) : 
    for i in tqdm(range(len(school))) : 
        distance = [] 
        for j in range(len(Police_station)) : 
            schoolloc = tuple(school[['경도','위도']].iloc[i])
            Police_stationloc = tuple(Police_station[['경도','위도']].iloc[j])
            distance.append(haversine(schoolloc, Police_stationloc, unit = 'm'))
            
        distance = [d for d in distance if d <= 500] 
        school[column][i] = len(distance)
        
    return school

In [12]:
school["500미터내_파출소개수"] = 0
school = close_ps(school, Police_station, "500미터내_파출소개수")

100%|████████████████████████████████████████████████████████████████████████████████| 320/320 [01:49<00:00,  2.91it/s]


In [13]:
school.groupby('500미터내_파출소개수').count()

,구,학교명,학교코드,내부상담_실시여부,외부상담_실시여부,wee클래스_운영여부,학교폭력_예방교육_평균교육시간(1학급당),위도,경도,법정동코드
500미터내_파출소개수,,,,,,,,,,
0,165,165,165,165,165,165,165,165,165,165
1,121,121,121,121,121,121,121,121,121,121
2,30,30,30,30,30,30,30,30,30,30
3,4,4,4,4,4,4,4,4,4,4


In [14]:
school

,구,학교명,학교코드,내부상담_실시여부,외부상담_실시여부,wee클래스_운영여부,학교폭력_예방교육_평균교육시간(1학급당),위도,경도,법정동코드,500미터내_파출소개수
0,강남구,개포고등학교,S010000373,1,1,1,382.4,37.484959,127.059405,1168010300,0
1,강남구,경기고등학교,S010000375,1,0,1,152.4,37.517566,127.056074,1168010500,1
2,강남구,경기여자고등학교,S010000376,1,1,1,200.0,37.486887,127.065567,1168010300,0
3,강남구,국립국악고등학교,S000003540,0,0,0,217.4,37.476230,127.051476,1168010300,1
4,강남구,단국대학교부속소프트웨어고등학교,S010000581,0,1,0,300.0,37.495617,127.057767,1168010600,0
...,...,...,...,...,...,...,...,...,...,...,...
315,성동구,무학여자고등학교,S010000436,1,1,1,253.3,37.556768,127.033599,1120010700,1
316,성동구,서울방송고등학교,S010000585,1,1,1,87.6,37.547773,127.011036,1120011300,1
317,성동구,성수고등학교,S010005523,1,1,1,406.6,37.547342,127.038253,1120011400,0
318,성동구,성수공업고등학교,S010000592,1,0,1,306.5,37.540540,127.051298,1120011500,1


In [15]:
school.to_csv('경감지수_전처리.csv', index=False)

## 구별 CCTV 대수 데이터

In [16]:
CCTV = pd.read_csv('서울시 CCTV 설치운영 현황(자치구)-목적별.csv',encoding='cp949', thousands = ',').iloc[:,-]
CCTV

,구분,총계,방범,어린이\n보호구역,공원·놀이터,쓰레기\n무단투기,시설안전·화재예방,교통단속,교통정보수집·분석,기타다른법령
0,총 계,75431,52359,7711,5942,1258,2012,5439,31,679
1,종로구,1729,1360,41,109,0,110,109,0,0
2,중구,2001,1180,75,165,62,62,290,8,159
3,용산구,2383,1740,164,154,0,0,252,0,73
4,성동구,3515,2834,0,177,151,15,338,0,0
5,광진구,2556,2011,301,98,71,5,70,0,0
6,동대문구,2315,1682,282,174,28,1,134,0,14
7,중랑구,3285,2872,141,141,11,54,66,0,0
8,성북구,3754,2489,400,192,359,108,183,23,0
9,강북구,2462,1831,319,187,0,32,93,0,0


In [17]:
CCTV = CCTV[['구분','방범','쓰레기\n무단투기']]
CCTV.head()

,구분,방범,쓰레기\n무단투기
0,총 계,52359,1258
1,종로구,1360,0
2,중구,1180,62
3,용산구,1740,0
4,성동구,2834,151


In [18]:
CCTV = CCTV.rename({'구분':'구','방범':'방범용CCTV','쓰레기\n무단투기':'어린이보호구역CCTV'},axis=1)
CCTV

,구,방범용CCTV,어린이보호구역CCTV
0,총 계,52359,1258
1,종로구,1360,0
2,중구,1180,62
3,용산구,1740,0
4,성동구,2834,151
5,광진구,2011,71
6,동대문구,1682,28
7,중랑구,2872,11
8,성북구,2489,359
9,강북구,1831,0


In [19]:
CCTV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   구            26 non-null     object
 1   방범용CCTV      26 non-null     int64 
 2   어린이보호구역CCTV  26 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 752.0+ bytes


- 데이터 merge

In [20]:
data_reduce = pd.merge(school,CCTV,on='구',how='left')
data_reduce.head()

,구,학교명,학교코드,내부상담_실시여부,외부상담_실시여부,wee클래스_운영여부,학교폭력_예방교육_평균교육시간(1학급당),위도,경도,법정동코드,500미터내_파출소개수,방범용CCTV,어린이보호구역CCTV
0,강남구,개포고등학교,S010000373,1,1,1,382.4,37.484959,127.059405,1168010300,0,4525,0
1,강남구,경기고등학교,S010000375,1,0,1,152.4,37.517566,127.056074,1168010500,1,4525,0
2,강남구,경기여자고등학교,S010000376,1,1,1,200.0,37.486887,127.065567,1168010300,0,4525,0
3,강남구,국립국악고등학교,S000003540,0,0,0,217.4,37.476230,127.051476,1168010300,1,4525,0
4,강남구,단국대학교부속소프트웨어고등학교,S010000581,0,1,0,300.0,37.495617,127.057767,1168010600,0,4525,0


In [21]:
data_reduce

,구,학교명,학교코드,내부상담_실시여부,외부상담_실시여부,wee클래스_운영여부,학교폭력_예방교육_평균교육시간(1학급당),위도,경도,법정동코드,500미터내_파출소개수,방범용CCTV,어린이보호구역CCTV
0,강남구,개포고등학교,S010000373,1,1,1,382.4,37.484959,127.059405,1168010300,0,4525,0
1,강남구,경기고등학교,S010000375,1,0,1,152.4,37.517566,127.056074,1168010500,1,4525,0
2,강남구,경기여자고등학교,S010000376,1,1,1,200.0,37.486887,127.065567,1168010300,0,4525,0
3,강남구,국립국악고등학교,S000003540,0,0,0,217.4,37.476230,127.051476,1168010300,1,4525,0
4,강남구,단국대학교부속소프트웨어고등학교,S010000581,0,1,0,300.0,37.495617,127.057767,1168010600,0,4525,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,성동구,무학여자고등학교,S010000436,1,1,1,253.3,37.556768,127.033599,1120010700,1,2834,151
316,성동구,서울방송고등학교,S010000585,1,1,1,87.6,37.547773,127.011036,1120011300,1,2834,151
317,성동구,성수고등학교,S010005523,1,1,1,406.6,37.547342,127.038253,1120011400,0,2834,151
318,성동구,성수공업고등학교,S010000592,1,0,1,306.5,37.540540,127.051298,1120011500,1,2834,151


In [22]:
data_reduce.isnull().sum()

구                         0
학교명                       0
학교코드                      0
내부상담_실시여부                 0
외부상담_실시여부                 0
wee클래스_운영여부               0
학교폭력_예방교육_평균교육시간(1학급당)    0
위도                        0
경도                        0
법정동코드                     0
500미터내_파출소개수              0
방범용CCTV                   0
어린이보호구역CCTV               0
dtype: int64

In [23]:
a=data_reduce[data_reduce['방범용CCTV'].isnull()].index
data_reduce.iloc[a]

,구,학교명,학교코드,내부상담_실시여부,외부상담_실시여부,wee클래스_운영여부,학교폭력_예방교육_평균교육시간(1학급당),위도,경도,법정동코드,500미터내_파출소개수,방범용CCTV,어린이보호구역CCTV


In [24]:
data_reduce.to_csv('경감지수_전처리2.csv', index=False)